# Scraping the two news sites with most coments, 20Min & Watson

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import twython

datestring = time.strftime("%Y-%h-%d")
datestring

'2016-Jul-09'

In [2]:
api_key = "Q5JK9VebduplKnM0JDHuAA7HD"
api_secret = "kOSrqYUlq1eTyX1fCoNtpS31zLMOdvA8vK5sppBP8OH6rD3yiW"

access_token = "751124626298765312-Cjgz95g9os1RNT6H3MaJCbdmagjI3YB"
token_secret = "RSdyXxAL3NTmpMa1AC6GlZ5PzphoaVULYnovFt6LceGAR"

In [3]:
twitter = twython.Twython(api_key, api_secret, access_token, token_secret)

In [4]:
# Grab the 20Min & Watson Homepages
response = requests.get("http://www.20min.ch")
response_watson = requests.get("http://www.watson.ch/")
response_Tagi = requests.get("http://www.tagesanzeiger.ch")

In [5]:
# Feed them into BeautifulSoup
_20Min = BeautifulSoup(response.text, 'html.parser')
Watson = BeautifulSoup(response_watson.text, 'html.parser')
Tagi = BeautifulSoup(response_Tagi.text, 'html.parser')

In [6]:
stories = _20Min.find_all("div", {'id': 'content'})
stories_watson = Watson.find_all('div', {'class': 'teaser teaser_type_default     font_m  combo_bg combo_fg'})
Tagi_stories = Tagi.find_all("div", {'class': 'featureStory standard'})
Tagi_kommentare = Tagi.find_all("div", {'class': 'articleStory valued'})

In [7]:
#finding everything outside of the loop
teaser_stories = stories[0].find_all('div', {'class': "teaser "})

#starting the dictionary:
all_stories = []

#The Loop
for story in teaser_stories:
    
    #Getting the comments
    comments = story.find('div', {'class': "teaser_media front clearfix"})
    if comments:
        try:
            comment = comments.find('span', {'class': 'icon_talkback'}).next_sibling
            #Just using the number of comments
            comment = comment[:4]
            #A regular expression to make sure to get articles with more of 999 comments
            if re.search(r"\D$", comment):
                comment = int(comment[:3])
            else:
                comment = int(comment)
        except:
            comment = 1
    
    #Getting the the title
    title_div = story.find('div', {'class': "teaser_title front"})
    title_string = title_div.find('h2').string
    if re.search(r"\x96", title_string):
        title_string.replace("\x96", "")
        
    
    #Getting the HREF
    href = story.find('h3')
    href = href.find('a', href=True)
    href = href['href']
    
    #Dealing with non-full links
    if re.search(r"^http:", href):
        href = href
    else:
        href = 'http://www.20min.ch' + href
    
    #Creating the dictionary:
    dictionary = {'Site': '@20min', 'Titel': title_string, 'Kommentare': int(comment), 'Link': href, 'Date': datestring}
    
    #List of dictionaries:
    all_stories.append(dictionary)

In [8]:
#starting the dictionary:
all_stories_watson = []

#looping through titles, comments, links
for story in stories_watson:
    
    #getting HREF
    href = story.find('a', {'class': 'storylink combo_bd'})
    try:
        href = href['href']
        href = ('http://watson.ch' + href)
    except:
        href == None
        
    #Getting the story
    title = story.find('h2')
    if title == None:
        title = None
    else:
        title = title.string
    
    #Getting the Comments
    comments = story.find('a', {'class': 'comments'})
    if comments == None:
        comments = 0
    else:
        comments = story.find('img', {'src': '/media/img/main/icons/icon_comments.png'}).span.text
        comments = int(comments)
    
    #creating the dictionary
    dictionary = {'Site': '@watson_news', 'Titel': title, 'Kommentare': int(comments), 'Link': href, 'Date': datestring}
    
    #creating the list
    all_stories_watson.append(dictionary)

In [9]:
Tagi_Dict = []

#looping through all the stories
for x in Tagi_stories:
    
    #Getting each headline and extracting only the text
    try:
        title = x.find('h3').get_text().strip()
    except:
        title == None

    #Getting the byline
    #time = title.find('p', {'class' : 'storyInfo'})
    
    #Getting time
    #time = x.find('p', {'class' : 'storyInfo'})
    #time = time.find('time', {'class' : 'time'})
    
    #Getting Kommentare
    if x.find('a', {'title' : 'Kommentare'}) is None:
        kommentare = 0
    else:
        kommentare = x.find('a', {'title' : 'Kommentare'}).string
        #kommentare = kommentare.replace(u'\xa0', ' ')
        kommentare = int(re.findall(r"\d+", kommentare)[0])
        
    #Getting HREF
    try:
        href = x.find('h3')
        href = href.find('a').get('href')
        href = 'http://www.tagesanzeiger.ch' + href
    except:
        href == None
        
    #Creating dictionary with Tagi Headlines, time
    title = { 'Site': '@tagesanzeiger', 'Titel': title, 'Kommentare': kommentare, 'Link': href, 'Date': datestring}
    
    Tagi_Dict.append(title)

In [10]:
Tagi_Kommentare_Dict = []

for x in Tagi_kommentare:
    
    #Getting each headline and extracting only the text
    try:
        title = x.find('h4').get_text().strip()
    except:
        title == None
    
    #Getting Kommentare
    if x.find('span', {'class' : 'icon icon-comment'}) is None:
        kommentare = 0
    else:
        kommentare = x.find('span', {'class' : 'icon icon-comment'}).next_sibling
        kommentare = kommentare[:4]
        #A regular expression to make sure to get articles with more of 999 comments
        if re.search(r"\D$", kommentare):
            kommentare = int(float(kommentare[:2]))
            print(kommentare)
        else:
            kommentare = int(kommentare)
    #Getting HREF
    try:
        href = x.find('h4')
        href = href.find('a').get('href')
        href = 'http://www.tagesanzeiger.ch' + href
    except:
        href == None
        
    #Creating dictionary with Tagi Headlines, time
    Kommentare = { 'Site': '@tagesanzeiger', 'Titel': title, 'Kommentare': kommentare, 'Link': href, 'Date': datestring}
    
    Tagi_Kommentare_Dict.append(Kommentare)

2


In [11]:
#adding the two dictionaries
all_stories.extend(all_stories_watson)
all_stories.extend(Tagi_Dict)
all_stories.extend(Tagi_Kommentare_Dict)

In [12]:
#sorting the dictionary according to the value of the keys
import operator
all_stories.sort(key=operator.itemgetter('Kommentare'), reverse=True)

In [13]:
print(all_stories)

[{'Titel': 'Nationalratskommission bittet Ältere zur Kasse', 'Site': '@20min', 'Kommentare': 687, 'Date': '2016-Jul-09', 'Link': 'http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162'}, {'Titel': 'Italienische Zahlstellen verwirren Reisende', 'Site': '@20min', 'Kommentare': 670, 'Date': '2016-Jul-09', 'Link': 'http://www.20min.ch/schweiz/news/story/Italienische-Zahlstellen-verwirren-Reisende-17793978'}, {'Titel': 'Lohnt es sich, den Motor am Rotlicht abzuschalten?', 'Site': '@20min', 'Kommentare': 590, 'Date': '2016-Jul-09', 'Link': 'http://www.20min.ch/schweiz/energy-challenge/story/Lohnt-es-sich--den-Motor-am-Rotlicht-abzuschalten--21177214'}, {'Titel': 'Krankenkassenprämien steigen immer mehr', 'Site': '@20min', 'Kommentare': 353, 'Date': '2016-Jul-09', 'Link': 'http://www.20min.ch/finance/news/story/Krankenkassenpraemien-steigen-immer-mehr-24655934'}, {'Titel': '99+ Bilder und Gifs gegen Langeweile: Der Picdump ist da!', 'Site': '@watson_ne

In [14]:
len(all_stories )
for x in all_stories:
    print(x['Site'], x['Titel'])

@20min Nationalratskommission bittet Ältere zur Kasse
@20min Italienische Zahlstellen verwirren Reisende
@20min Lohnt es sich, den Motor am Rotlicht abzuschalten?
@20min Krankenkassenprämien steigen immer mehr
@watson_news 99+ Bilder und Gifs gegen Langeweile: Der Picdump ist da!
@20min «Ich bin wütend und traurig»
@20min «So sehen Bünzli aus, die am Rasenmähen sind»
@20min Salt-Abos fliegen bei Mobilezone raus
@20min «Ich bin nach einer Harry Potter-Figur benannt»
@20min Was kann man tun, wenn ein Flug annulliert wird?
@20min Darum hatte Salt gestern einen Totalausfall
@20min Bomben-Roboter ist die neue Waffe der Polizei
@20min Rührende Szenen nach dem Schlusspfiff
@watson_news EM-Tagesticker: Die spezielle Geschichte von Raphael Guerreiros +++ welcher Torjäger drückt dem Final seinen Stempel auf?
@20min Finden Sie heraus, wie Sie mit dem Chef reden sollten
@20min UPC-Kunden müssen ihr Passwort ändern
@20min «Die Zustände in den Camps gehen mir nahe»
@watson_news Weil ihr so schön b

In [15]:
all_stories[0]

{'Date': '2016-Jul-09',
 'Kommentare': 687,
 'Link': 'http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162',
 'Site': '@20min',
 'Titel': 'Nationalratskommission bittet Ältere zur Kasse'}

In [16]:
all_stories[0]['Titel']

'Nationalratskommission bittet Ältere zur Kasse'

In [17]:
len(all_stories)

181

In [18]:
Sum = 0
for item in all_stories:
    Sum = Sum + item['Kommentare']

In [19]:
Sum

6508

In [20]:
Kommentar_Schnitt = round(Sum / len(all_stories))

In [21]:
Count = all_stories[0]['Kommentare']
#if Count > 1100:
#    Comment_Count = "Lesen. Dazu hat Krethi und Plethi eine Meinung."
#elif Count > 900:
#    Comment_Count = "Party-Talk. Wirklich an jeder Party."
#elif Count > 700:
#    Comment_Count = "Da haben am Stammtisch einige etwas dazu zu sagen."
#elif Count > 500:
#    Comment_Count = "Das bewegt einge. Am richtig viele sind das noch nicht."
#elif Count > 0:
#    Comment_Count = "Da schlafen einige fleissige Kommentarschreiben wohl noch."

In [22]:
#sentence = 'Kommentare: {}. Quelle: {}. {} {}'.format(all_stories[0]['Kommentare'], all_stories[0]['Site'], Comment_Count, all_stories[0]['Link'])

In [23]:
sentence = 'Die heisseste Front-Story läuft {} mit {} Kommentaren. Der Schnitt liegt bei {} Reaktionen pro Story. {}'.format(all_stories[0]['Site'], all_stories[0]['Kommentare'], Kommentar_Schnitt, all_stories[0]['Link']) 

print(sentence)

In [25]:
#twitter = twython.Twython(api_key, api_secret)
#auth = twitter.get_authentication_tokens()
#print("Log into Twitter as the user you want to authorize and visit this URL:")
#print("\t" + auth['auth_url'])

In [26]:
#pin = "XXXXXX"
#
#twitter = twython.Twython(api_key, api_secret, auth['oauth_token'], auth['oauth_token_secret'])
#tokens = twitter.get_authorized_tokens(pin)
#
#new_access_token = tokens['oauth_token']
#new_token_secret = tokens['oauth_token_secret']
#print("your access token:", new_access_token)
#print("your token secret:", new_token_secret)

In [27]:
twitter.update_status(status=sentence)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jul 09 16:21:43 +0000 2016',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': '20min.ch/schweiz/news/s…',
    'expanded_url': 'http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162',
    'indices': [110, 133],
    'url': 'https://t.co/xkafvQCw7W'}],
  'user_mentions': [{'id': 15786941,
    'id_str': '15786941',
    'indices': [32, 38],
    'name': '20 Minuten',
    'screen_name': '20min'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 751813593478356993,
 'id_str': '751813593478356993',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'de',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.barnabyskinner.com/" rel="nof